In [107]:
import requests
import random
from datetime import datetime, timedelta, time
import math
# import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text

In [108]:
class Waktu:
    def __init__(self, hari, jam_mulai, jam_selesai):
        self.hari = hari

        self.jam_mulai = jam_mulai
        self.jam_selesai = jam_selesai
    def __repr__(self):
        return f"Waktu(hari={self.hari}, jam_mulai={self.jam_mulai}, jam_selesai={self.jam_selesai})"

In [109]:
    
class Ruang:
    def __init__(self, nama, tipe_ruang):
        self.nama = nama
        self.tipe_ruang = tipe_ruang
    
    def __repr__(self):
        return f"Ruang(nama={self.nama}, tipe_ruang={self.tipe_ruang})"

In [110]:
class Matakuliah:
    def __init__(self, matkul, dosen, sks, kelas, status, id_perkuliahan, id_semester,kategori): #tambah id_perkuliahan
        self.id_perkuliahan = id_perkuliahan
        self.matkul = matkul
        self.dosen = dosen
        self.sks = sks
        self.status = status
        self.kelas = kelas
        self.id_semester = id_semester
        self.kategori = kategori
        self.ruang_needed = self.set_ruang(kategori)

    def __repr__(self):
        return (f"Matakuliah(matkul={self.matkul}, dosen={self.dosen}, sks={self.sks}, status={self.status})")
    
    def set_ruang(self, kategori):
        if kategori == "Teori":
            return ["Kelas"]
        elif kategori == "Praktikum":
            return ["Lab"]
        elif kategori == "Gabungan":
            return ["Kelas", "Lab"]
        else:
            return []

In [111]:
class PenjadwalanSA:
    def __init__(self):
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = []
        self.generate_slot_waktu() 
        print(f"Jumlah slot waktu: {len(self.daftar_slot)}")

    def generate_slot_waktu(self):
        self.daftar_slot.clear()
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    # Slot tumpang tindih dengan istirahat
                    waktu_mulai = istirahat_selesai  # lanjut setelah istirahat
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                self.daftar_slot.append((waktu_mulai.strftime("%H:%M"), waktu_selesai.strftime("%H:%M")))
                waktu_mulai = waktu_selesai

sa = PenjadwalanSA()
print(sa.daftar_slot)




Jumlah slot waktu: 13
[('07:00', '07:50'), ('07:50', '08:40'), ('08:40', '09:30'), ('09:45', '10:35'), ('10:35', '11:25'), ('11:25', '12:15'), ('12:45', '13:35'), ('13:35', '14:25'), ('14:25', '15:15'), ('15:30', '16:20'), ('16:20', '17:10'), ('17:10', '18:00'), ('18:15', '19:05')]


In [113]:
class PenjadwalanSA:
    def __init__(self, initial_temperature=10000, cooling_rate=0.9999, max_iterations=100000):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        
        # Data penjadwalan
        self.daftar_dosen = []
        self.daftar_ruang = []
        self.daftar_matkul = []
        self.daftar_hari = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
        

        # Slot waktu
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = []
    
        # self.setDataCsv()

        self.baca_datamk()
        # self.get_ruang()
        self.baca_dataruang()

        # Generate slot waktu awal
        self.generate_slot_waktu() 

    def baca_datamk(self):
        # baca dari database tabel perkuliahan dengan id generate 1 --belum
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
            SELECT tb_rombel.nama_kelas AS kelas,
                tb_matakuliah.nama_matakuliah AS matakuliah,
                tb_dosen.nama_dosen AS dosen,
                tb_matakuliah.sks AS sks,
                tb_matakuliah.status AS status,
                tb_matakuliah.kategori AS kategori,
                id_perkuliahan,
                id_semester
            FROM tb_perkuliahan
            JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
            JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
            JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
            JOIN tb_generate 
            ON tb_generate.id_generate = tb_rombel.id_generate 
            AND tb_generate.id_generate = tb_matakuliah.id_generate 
            AND tb_generate.id_generate = tb_dosen.id_generate
            AND tb_generate.id_generate = 'G0706-02'
            LIMIT 40
        """)
        with engine.connect() as connection:
            df_matkul = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_matkul.iterrows():
            self.tambah_matkul(row['matakuliah'], row['dosen'], row['sks'], row['kelas'], row['status'], row['id_perkuliahan'],row['id_semester'],row['kategori'])
            

    def baca_dataruang(self):
        # berasal dari database dengan id generate 1
        # Iterasi per baris untuk menambahkan data
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        AND tb_generate.id_generate = 'G0706-02'
        LIMIT 40
        """)
        with engine.connect() as connection:
            df_ruang = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_ruang.iterrows():
            self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
            
        
    def tambah_dosen(self, nama):
        dosen = Dosen(nama)
        self.daftar_dosen.append(dosen)
        return dosen
    
    def tambah_ruang(self, nama, tiperuang):
        ruang = Ruang(nama,tiperuang)
        self.daftar_ruang.append(ruang)
        return ruang
    
    def tambah_matkul(self,matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,kategori):
        matkul = Matakuliah(matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,kategori) #tambahkan id_perkuliahan
        self.daftar_matkul.append(matkul)
        return matkul
    
    def tambah_hari(self,hari):
        hari = Hari(hari)
        self.daftar_hari.append(hari)
        return hari

    def generate_slot_waktu(self):
        self.daftar_slot.clear()
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    # Slot tumpang tindih dengan istirahat
                    waktu_mulai = istirahat_selesai  # lanjut setelah istirahat
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                self.daftar_slot.append((waktu_mulai.strftime("%H:%M"), waktu_selesai.strftime("%H:%M")))
                waktu_mulai = waktu_selesai


    def hitung_sks(self, sks, kategori):
        if kategori == "Teori":
            return sks 
        elif kategori == "Praktikum":
            return sks * 3
        elif kategori == "Gabungan":
            return 7 
        return  0
    
    def get_ruang(self, matkul):
        ruang_sesuai = [
            r for r in self.daftar_ruang
            if not matkul.ruang_needed or r.tipe_ruang in matkul.ruang_needed
        ]
        return random.choice(ruang_sesuai) if ruang_sesuai else random.choice(self.daftar_ruang)
            
    def solusi_awal(self):
        jadwal_awal = []
        self.generate_slot_waktu()

        class_schedule = {}  # Menyimpan jadwal kelas yang sudah ada

        for matkul in self.daftar_matkul:
            ruang = self.get_ruang(matkul)
            hari = random.randint(0, len(self.daftar_hari) - 1)

            sukses = False
            durasi_slot = self.hitung_sks(matkul.sks, matkul.kategori)

            for _ in range(100):  # Lebih banyak percobaan
                waktu_mulai = random.randint(0, len(self.daftar_slot) - durasi_slot)
                waktu_selesai = waktu_mulai + durasi_slot

                class_key = (matkul.kelas, hari)
                if class_key not in class_schedule:
                    class_schedule[class_key] = []

                conflict = False
                for existing_mulai, existing_selesai in class_schedule[class_key]:
                    if not (waktu_selesai <= existing_mulai or waktu_mulai >= existing_selesai):
                        conflict = True
                        break

                if not conflict:
                    class_schedule[class_key].append((waktu_mulai, waktu_selesai))
                    jadwal_awal.append((matkul, ruang, hari, waktu_mulai))
                    sukses = True
                    break

            if not sukses:
                jadwal_awal.append((matkul, ruang, hari, -1))  # Gagal penempatan

        return jadwal_awal

    def get_neighbor(self, solution):
        neighbor = solution.copy()

        if len(neighbor) < 2:
            # Jika cuma 1 atau 0 jadwal, hanya pindah saja
            i = 0
            matkul, _, _, _ = neighbor[i]
            ruang = self.get_ruang(matkul)
            hari = random.randint(0, len(self.daftar_hari) - 1)
            durasi = self.hitung_sks(matkul.sks, matkul.kategori)
            jam_mulai = random.randint(0, len(self.daftar_slot) - durasi)
            neighbor[i] = (matkul, ruang, hari, jam_mulai)
            return neighbor

        if random.random() < 0.5:
            # MOVE: ubah jadwal satu mata kuliah
            i = random.randint(0, len(neighbor) - 1)
            matkul, _, _, _ = neighbor[i]
            ruang = self.get_ruang(matkul)
            hari = random.randint(0, len(self.daftar_hari) - 1)
            durasi = self.hitung_sks(matkul.sks, matkul.kategori)
            jam_mulai = random.randint(0, len(self.daftar_slot) - durasi)
            neighbor[i] = (matkul, ruang, hari, jam_mulai)
        else:
            # SWAP: tukar jadwal dua mata kuliah
            i, j = random.sample(range(len(neighbor)), 2)
            matkul_i, ruang_i, hari_i, jam_i = neighbor[i]
            matkul_j, ruang_j, hari_j, jam_j = neighbor[j]
            neighbor[i] = (matkul_i, ruang_j, hari_j, jam_j)
            neighbor[j] = (matkul_j, ruang_i, hari_i, jam_i)

        return neighbor
    
    def evaluate_solution(self, solution):
        score = 0

        kelas_jadwal = {}
        dosen_jadwal = {}
        ruang_jadwal = {}
        kelas_hari_map = {}

        for matkul, ruang, hari, jam_mulai in solution:
            durasi = self.hitung_sks(matkul.sks, matkul.kategori)
            jam_selesai = jam_mulai + durasi

            if jam_mulai == -1:
                score += 10  # penalti jika tidak bisa dijadwalkan
                continue

            for jam in range(jam_mulai, jam_selesai):
                # Cek bentrok di kelas
                key_kelas = (matkul.kelas, hari, jam)
                if key_kelas in kelas_jadwal:
                    score += 5
                else:
                    kelas_jadwal[key_kelas] = matkul

                # Cek bentrok di dosen
                key_dosen = (matkul.dosen, hari, jam)
                if key_dosen in dosen_jadwal:
                    score += 5
                else:
                    dosen_jadwal[key_dosen] = matkul

                # Cek bentrok di ruang
                key_ruang = (ruang.nama, hari, jam)
                if key_ruang in ruang_jadwal:
                    score += 5
                else:
                    ruang_jadwal[key_ruang] = matkul

            # Simpan jadwal untuk validasi tumpang tindih dalam kelas
            key_kelas_hari = (matkul.kelas, hari)
            if key_kelas_hari not in kelas_hari_map:
                kelas_hari_map[key_kelas_hari] = []
            kelas_hari_map[key_kelas_hari].append((jam_mulai, jam_selesai))

        # Validasi tumpang tindih dalam kelas (urutan waktu)
        for timeslots in kelas_hari_map.values():
            timeslots.sort()
            for i in range(len(timeslots) - 1):
                if timeslots[i][1] > timeslots[i + 1][0]:
                    score += 10  # penalti untuk tumpang tindih

        return score
    
    def anneal(self):
        current_solution = self.solusi_awal()
        current_score = self.evaluate_solution(current_solution)
        best_solution, best_score = current_solution, current_score
        temperature = self.initial_temperature

        for iteration in range(self.max_iterations):
            if temperature <= 0:
                break

            neighbor_solution = self.get_neighbor(current_solution)
            neighbor_score = self.evaluate_solution(neighbor_solution)

            delta_score = neighbor_score - current_score
            acceptance_probability = math.exp(-delta_score / temperature) if delta_score > 0 else 1

            if random.random() < acceptance_probability:
                current_solution, current_score = neighbor_solution, neighbor_score
                if current_score < best_score:
                    best_solution, best_score = current_solution, current_score

            temperature *= self.cooling_rate
            if iteration % 10000 == 0:
                print(f"Iterasi {iteration} | Skor saat ini: {current_score} | Skor terbaik: {best_score} | Temperatur: {temperature:.4f}")

        return best_solution, best_score
    
    def tampilkan_jadwal(self, solution):
        jadwal_terjadwal = {}

        for matkul, ruang, hari, waktu in solution:
            if hari not in jadwal_terjadwal:
                jadwal_terjadwal[hari] = []
            slot_awal = waktu
            slot_akhir = waktu + matkul.sks
            jadwal_terjadwal[hari].append((matkul, ruang, (slot_awal, slot_akhir)))

        for hari, jadwal in sorted(jadwal_terjadwal.items()):
            print(f"\nHari: {hari}")
            for matkul, ruang, waktu in sorted(jadwal, key=lambda x: x[2][0]):
                waktu_mulai = self.daftar_slot[waktu[0]][0]
                waktu_selesai = self.daftar_slot[waktu[1] - 1][1]
                print(f"{waktu_mulai} - {waktu_selesai} | {matkul.matkul} ({matkul.sks} SKS) | {ruang.nama} | Dosen: {matkul.dosen}")


    def df_jadwaloptimasi(self, solution):
        data = []
        for matkul, ruang, hari, waktu in solution:
            waktu_mulai = self.daftar_slot[waktu[0]][0]  # Ambil waktu mulai dari daftar slot
            waktu_selesai = self.daftar_slot[waktu[1] - 1][1]  # Ambil waktu selesai dari daftar slot

            data.append({
                "id_perkuliahan" : matkul.id_perkuliahan,
                "hari": hari,
                "waktu_mulai": waktu_mulai,
                "waktu_selesai": waktu_selesai,
                "kelas": matkul.kelas,
                "mata_kuliah": matkul.matkul,
                "nama_dosen": matkul.dosen,
                "ruang": ruang.nama,
                "semester":matkul.id_semester,
                
            })

        df_jadwal = pd.DataFrame(data)
        return df_jadwal
    
    def cek_bentrok(self, solution):
        def is_overlap(start1, end1, start2, end2):
            return not (end1 <= start2 or start1 >= end2)

        konflik_ruang = []
        konflik_dosen = []
        konflik_kelas = []

        slot_ruang = {}
        slot_dosen = {}
        slot_kelas = {}

        for matkul, ruang, hari, waktu in solution:
            waktu_mulai, waktu_selesai = waktu
            waktu_str = f"{self.daftar_slot[waktu_mulai][0]} - {self.daftar_slot[waktu_selesai - 1][1]}"

            # Bentrok Ruang
            key_ruang = (ruang.nama, hari)
            if key_ruang not in slot_ruang:
                slot_ruang[key_ruang] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_ruang[key_ruang]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_ruang.append({
                        "ruang": ruang.nama, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul, "kelas": matkul.kelas,
                        "matkul_bentrok": existing_matkul.matkul, "kelas_bentrok": existing_matkul.kelas
                    })
            slot_ruang[key_ruang].append((waktu_mulai, waktu_selesai, matkul))

            # Bentrok Dosen
            key_dosen = (matkul.dosen, hari)
            if key_dosen not in slot_dosen:
                slot_dosen[key_dosen] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_dosen[key_dosen]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_dosen.append({
                        "dosen": matkul.dosen, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul, "kelas": matkul.kelas,
                        "matkul_bentrok": existing_matkul.matkul, "kelas_bentrok": existing_matkul.kelas
                    })
            slot_dosen[key_dosen].append((waktu_mulai, waktu_selesai, matkul))

            # Bentrok Kelas
            key_kelas = (matkul.kelas, hari)
            if key_kelas not in slot_kelas:
                slot_kelas[key_kelas] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_kelas[key_kelas]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_kelas.append({
                        "kelas": matkul.kelas, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul,
                        "matkul_bentrok": existing_matkul.matkul
                    })
            slot_kelas[key_kelas].append((waktu_mulai, waktu_selesai, matkul))

        # === TAMPILKAN HASIL KONFLIK ===
        def tampilkan_konflik(judul, konflik_list, keterangan):
            if konflik_list:
                print(f"\n❌ {judul}:")
                for konflik in konflik_list:
                    print(f"  {keterangan.format(**konflik)}")
            else:
                print(f"\n✅ Tidak ada {judul.lower()}.")

        tampilkan_konflik(
            "Konflik Ruang", konflik_ruang,
            "Ruang: {ruang} | Hari: {hari} | Waktu: {waktu}\n    ↪ {kelas} - {matkul} ❌ bentrok dengan {kelas_bentrok} - {matkul_bentrok}"
        )
        tampilkan_konflik(
            "Konflik Dosen", konflik_dosen,
            "Dosen: {dosen} | Hari: {hari} | Waktu: {waktu}\n    ↪ {kelas} - {matkul} ❌ bentrok dengan {kelas_bentrok} - {matkul_bentrok}"
        )
        tampilkan_konflik(
            "Konflik Kelas", konflik_kelas,
            "Kelas: {kelas} | Hari: {hari} | Waktu: {waktu}\n    ↪ {matkul} ❌ bentrok dengan {matkul_bentrok}"
        )

    def jumlah_jadwal_gagal(self, solution):
        return sum(1 for _, _, _, waktu in solution if waktu == (-1, -1))



sa = PenjadwalanSA()
best_solution, best_score = sa.anneal()
print("Skor akhir:", best_score)
print("Jumlah jadwal gagal:", sa.jumlah_jadwal_gagal(best_solution), "dari total", len(sa.daftar_matkul))

sa.tampilkan_jadwal(best_solution)
sa.cek_bentrok(best_solution)
# sa.simpan_optimasi(df_jadwal)

Iterasi 0 | Skor saat ini: 225 | Skor terbaik: 175 | Temperatur: 9999.0000
Iterasi 10000 | Skor saat ini: 725 | Skor terbaik: 175 | Temperatur: 3678.2426
Iterasi 20000 | Skor saat ini: 720 | Skor terbaik: 175 | Temperatur: 1353.0822
Iterasi 30000 | Skor saat ini: 730 | Skor terbaik: 175 | Temperatur: 497.7462
Iterasi 40000 | Skor saat ini: 715 | Skor terbaik: 175 | Temperatur: 183.1014
Iterasi 50000 | Skor saat ini: 615 | Skor terbaik: 175 | Temperatur: 67.3559
Iterasi 60000 | Skor saat ini: 545 | Skor terbaik: 175 | Temperatur: 24.7776
Iterasi 70000 | Skor saat ini: 430 | Skor terbaik: 175 | Temperatur: 9.1147
Iterasi 80000 | Skor saat ini: 225 | Skor terbaik: 175 | Temperatur: 3.3529
Iterasi 90000 | Skor saat ini: 155 | Skor terbaik: 155 | Temperatur: 1.2334
Skor akhir: 150
Jumlah jadwal gagal: 0 dari total 40

Hari: 0
07:00 - 08:40 | Praktikum Keamanan Jaringan  (2 SKS) | Ruang 313 | Dosen: Hendrik Kusbandono, S.Kom., M.Kom
07:00 - 08:40 | Digital Entrepreneurship (2 SKS) | Ruang 31

IndexError: list index out of range

In [120]:
class PenjadwalanSA:
    def __init__(self, initial_temperature=100000, cooling_rate=0.99999, max_iterations=1000000):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        
        # Data penjadwalan
        self.daftar_dosen = []
        self.daftar_ruang = []
        self.daftar_matkul = []
        self.daftar_hari = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
        

        # Slot waktu
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = []
    
        # self.setDataCsv()

        self.baca_datamk()
        # self.get_ruang()
        self.baca_dataruang()

        # Generate slot waktu awal
        self.generate_slot_waktu() 

    def baca_datamk(self):
        # baca dari database tabel perkuliahan dengan id generate 1 --belum
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
            SELECT tb_rombel.nama_kelas AS kelas,
                tb_matakuliah.nama_matakuliah AS matakuliah,
                tb_dosen.nama_dosen AS dosen,
                tb_matakuliah.sks AS sks,
                tb_matakuliah.status AS status,
                tb_matakuliah.kategori AS kategori,
                id_perkuliahan,
                id_semester
            FROM tb_perkuliahan
            JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
            JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
            JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
            JOIN tb_generate 
            ON tb_generate.id_generate = tb_rombel.id_generate 
            AND tb_generate.id_generate = tb_matakuliah.id_generate 
            AND tb_generate.id_generate = tb_dosen.id_generate
            AND tb_generate.id_generate = 'G0706-02'
            LIMIT 40
        """)
        with engine.connect() as connection:
            df_matkul = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_matkul.iterrows():
            self.tambah_matkul(row['matakuliah'], row['dosen'], row['sks'], row['kelas'], row['status'], row['id_perkuliahan'],row['id_semester'],row['kategori'])
            

    def baca_dataruang(self):
        # berasal dari database dengan id generate 1
        # Iterasi per baris untuk menambahkan data
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        AND tb_generate.id_generate = 'G0706-02'
        LIMIT 40
        """)
        with engine.connect() as connection:
            df_ruang = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_ruang.iterrows():
            self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
            
        
    def tambah_dosen(self, nama):
        dosen = Dosen(nama)
        self.daftar_dosen.append(dosen)
        return dosen
    
    def tambah_ruang(self, nama, tiperuang):
        ruang = Ruang(nama,tiperuang)
        self.daftar_ruang.append(ruang)
        return ruang
    
    def tambah_matkul(self,matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,kategori):
        matkul = Matakuliah(matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,kategori) #tambahkan id_perkuliahan
        self.daftar_matkul.append(matkul)
        return matkul
    
    def tambah_hari(self,hari):
        hari = Hari(hari)
        self.daftar_hari.append(hari)
        return hari

    def generate_slot_waktu(self):
        self.daftar_slot.clear()
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    # Slot tumpang tindih dengan istirahat
                    waktu_mulai = istirahat_selesai  # lanjut setelah istirahat
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                self.daftar_slot.append((waktu_mulai.strftime("%H:%M"), waktu_selesai.strftime("%H:%M")))
                waktu_mulai = waktu_selesai


    def hitung_sks(self, sks, kategori):
        if kategori == "Teori":
            return sks 
        elif kategori == "Praktikum":
            return sks * 3
        elif kategori == "Gabungan":
            return 7 
        return  0
    
    def get_ruang(self, kategori):
        if kategori == "Teori":
            return {"Kelas"} 
        elif kategori == "Praktikum":
            return {"Lab"} 
        elif kategori == "Gabungan":
            return {"Lab"}  
        return set()
    
    def solusi_awal(self):
        jadwal_awal = []
        self.generate_slot_waktu()
        
        class_schedule = {}  # Menyimpan jadwal kelas yang sudah ada
        
        for matkul in self.daftar_matkul:
            tipe_valid = self.get_ruang(matkul.kategori)
            ruang_valid = [ruang for ruang in self.daftar_ruang if ruang.tipe_ruang in tipe_valid]
            if ruang_valid:
                ruang = random.choice(ruang_valid)
                hari = random.choice(self.daftar_hari)
                
                sukses = False
                for _ in range(1000):  # Dulu 100, sekarang 1000 agar lebih banyak mencoba
                    waktu_mulai = random.randint(0, len(self.daftar_slot) - self.hitung_sks(matkul.sks, matkul.kategori))
                    waktu_selesai = waktu_mulai + self.hitung_sks(matkul.sks, matkul.kategori)
                    waktu = (waktu_mulai, waktu_selesai)

                    class_key = (matkul.kelas, hari)
                    if class_key not in class_schedule:
                        class_schedule[class_key] = []

                    conflict = False
                    for existing_slot in class_schedule[class_key]:
                        if not (waktu_selesai <= existing_slot[0] or waktu_mulai >= existing_slot[1]):
                            conflict = True
                            break

                    if not conflict:
                        class_schedule[class_key].append(waktu)
                        jadwal_awal.append((matkul, ruang, hari, waktu))
                        sukses = True
                        break

                if not sukses:
                    # Tandai sebagai gagal dijadwalkan
                    jadwal_awal.append((matkul, ruang, hari, (-1, -1)))
        return jadwal_awal

    def evaluate_solution(self, solution):
        score = 0
        slot_used = {}
        class_schedule = {}

        for matkul, ruang, hari, waktu in solution:
            if waktu == (-1, -1):
                score += 1000  # Penalti sangat tinggi agar SA memprioritaskan perbaikannya
                continue

            slot_key = (ruang.nama, hari, waktu)
            class_key = (matkul.kelas, hari)

            # Cek konflik ruang
            if slot_key in slot_used:
                score += 5
            else:
                slot_used[slot_key] = True

            # Cek konflik jadwal kelas
            if class_key not in class_schedule:
                class_schedule[class_key] = []
            
            waktu_mulai, waktu_selesai = waktu
            conflict = False
            for existing_start, existing_end in class_schedule[class_key]:
                if not (waktu_selesai <= existing_start or waktu_mulai >= existing_end):
                    conflict = True
                    break

            if conflict:
                score += 10
            else:
                class_schedule[class_key].append((waktu_mulai, waktu_selesai))

        return score


    def get_neighbor(self, solution):
        neighbor = solution[:]

        # Prioritaskan yang gagal dijadwalkan (waktu == (-1, -1))
        index_gagal = [i for i, (_, _, _, waktu) in enumerate(neighbor) if waktu == (-1, -1)]

        if index_gagal:
            idx = random.choice(index_gagal)
        else:
            idx = random.randint(0, len(solution) - 1)

        matkul, _, _, _ = neighbor[idx]

        # Cari ruang yang sesuai
        tipe_valid = self.get_ruang(matkul.kategori)
        ruang_valid = [r for r in self.daftar_ruang if r.tipe_ruang in tipe_valid]
        
        if not ruang_valid:
            return neighbor  # Tidak ada ruang valid, tidak bisa perbaiki

        ruang = random.choice(ruang_valid)
        hari = random.choice(self.daftar_hari)

        max_start_index = len(self.daftar_slot) - self.hitung_sks(matkul.sks, matkul.kategori)
        if max_start_index <= 0:
            waktu = (-1, -1)
        else:
            waktu_mulai = random.randint(0, max_start_index)
            waktu_selesai = waktu_mulai + self.hitung_sks(matkul.sks, matkul.kategori)
            waktu = (waktu_mulai, waktu_selesai)

        # Perbarui
        neighbor[idx] = (matkul, ruang, hari, waktu)
        return neighbor


    def anneal(self):
        current_solution = self.solusi_awal()
        current_score = self.evaluate_solution(current_solution)
        best_solution, best_score = current_solution, current_score
        temperature = self.initial_temperature

        for iteration in range(self.max_iterations):
            if temperature <= 0:
                break

            neighbor_solution = self.get_neighbor(current_solution)
            neighbor_score = self.evaluate_solution(neighbor_solution)

            delta_score = neighbor_score - current_score
            acceptance_probability = math.exp(-delta_score / temperature) if delta_score > 0 else 1

            if random.random() < acceptance_probability:
                current_solution, current_score = neighbor_solution, neighbor_score
                if current_score < best_score:
                    best_solution, best_score = current_solution, current_score

            temperature *= self.cooling_rate
            if iteration % 10000 == 0:
                print(f"Iterasi {iteration} | Skor saat ini: {current_score} | Skor terbaik: {best_score} | Temperatur: {temperature:.4f}")

        return best_solution, best_score

    def tampilkan_jadwal(self, solution):
        jadwal_terjadwal = {}

        for matkul, ruang, hari, waktu in solution:
            if hari not in jadwal_terjadwal:
                jadwal_terjadwal[hari] = []
            jadwal_terjadwal[hari].append((matkul, ruang, waktu))

        for hari, jadwal in sorted(jadwal_terjadwal.items()):
            print(f"\nHari: {hari}")
            for matkul, ruang, waktu in sorted(jadwal, key=lambda x: x[2][0]):  # Urutkan berdasarkan waktu mulai
                waktu_mulai = self.daftar_slot[waktu[0]][0]  # Ambil waktu mulai dari daftar slot
                waktu_selesai = self.daftar_slot[waktu[1] - 1][1]  # Ambil waktu selesai dari daftar slot
                
                print(f"  Slot: {waktu_mulai} - {waktu_selesai}, "
                    f"Kelas: {matkul.kelas}, "
                    f"Mata Kuliah: {matkul.matkul}, "
                    f"Dosen: {matkul.dosen}, "
                    f"Ruang: {ruang.nama}")
    def tampilkan_slot_waktu(self):
        print("Daftar Slot Waktu:")
        for i, slot in enumerate(self.daftar_slot, 1):
            print(f"{i}. {slot[0]} - {slot[1]}")

        
    # def simpan_optimasi(engine, df, table_name='tb_hasil'):
    #     db = 'mysql+pymysql://root:@localhost/db_optimasi1'
    #     engine = create_engine(db)

    #     df.to_sql(table_name, con=engine, if_exists='append', index=False)
    #     print(f'Data berhasil disimpan ke database {table_name}')
    
    def df_jadwaloptimasi(self, solution):
        data = []
        for matkul, ruang, hari, waktu in solution:
            waktu_mulai = self.daftar_slot[waktu[0]][0]  # Ambil waktu mulai dari daftar slot
            waktu_selesai = self.daftar_slot[waktu[1] - 1][1]  # Ambil waktu selesai dari daftar slot

            data.append({
                "id_perkuliahan" : matkul.id_perkuliahan,
                "hari": hari,
                "waktu_mulai": waktu_mulai,
                "waktu_selesai": waktu_selesai,
                "kelas": matkul.kelas,
                "mata_kuliah": matkul.matkul,
                "nama_dosen": matkul.dosen,
                "ruang": ruang.nama,
                "semester":matkul.id_semester,
                
            })

        df_jadwal = pd.DataFrame(data)
        return df_jadwal
    
    def cek_bentrok(self, solution):
        def is_overlap(start1, end1, start2, end2):
            return not (end1 <= start2 or start1 >= end2)

        konflik_ruang = []
        konflik_dosen = []
        konflik_kelas = []

        slot_ruang = {}
        slot_dosen = {}
        slot_kelas = {}

        for matkul, ruang, hari, waktu in solution:
            waktu_mulai, waktu_selesai = waktu
            waktu_str = f"{self.daftar_slot[waktu_mulai][0]} - {self.daftar_slot[waktu_selesai - 1][1]}"

            # Bentrok Ruang
            key_ruang = (ruang.nama, hari)
            if key_ruang not in slot_ruang:
                slot_ruang[key_ruang] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_ruang[key_ruang]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_ruang.append({
                        "ruang": ruang.nama, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul, "kelas": matkul.kelas,
                        "matkul_bentrok": existing_matkul.matkul, "kelas_bentrok": existing_matkul.kelas
                    })
            slot_ruang[key_ruang].append((waktu_mulai, waktu_selesai, matkul))

            # Bentrok Dosen
            key_dosen = (matkul.dosen, hari)
            if key_dosen not in slot_dosen:
                slot_dosen[key_dosen] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_dosen[key_dosen]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_dosen.append({
                        "dosen": matkul.dosen, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul, "kelas": matkul.kelas,
                        "matkul_bentrok": existing_matkul.matkul, "kelas_bentrok": existing_matkul.kelas
                    })
            slot_dosen[key_dosen].append((waktu_mulai, waktu_selesai, matkul))

            # Bentrok Kelas
            key_kelas = (matkul.kelas, hari)
            if key_kelas not in slot_kelas:
                slot_kelas[key_kelas] = []
            for existing_mulai, existing_selesai, existing_matkul in slot_kelas[key_kelas]:
                if is_overlap(waktu_mulai, waktu_selesai, existing_mulai, existing_selesai):
                    konflik_kelas.append({
                        "kelas": matkul.kelas, "hari": hari, "waktu": waktu_str,
                        "matkul": matkul.matkul,
                        "matkul_bentrok": existing_matkul.matkul
                    })
            slot_kelas[key_kelas].append((waktu_mulai, waktu_selesai, matkul))

        # === TAMPILKAN HASIL KONFLIK ===
        def tampilkan_konflik(judul, konflik_list, keterangan):
            if konflik_list:
                print(f"\n❌ {judul}:")
                for konflik in konflik_list:
                    print(f"  {keterangan.format(**konflik)}")
            else:
                print(f"\n✅ Tidak ada {judul.lower()}.")

        tampilkan_konflik(
            "Konflik Ruang", konflik_ruang,
            "Ruang: {ruang} | Hari: {hari} | Waktu: {waktu}\n    ↪ {kelas} - {matkul} ❌ bentrok dengan {kelas_bentrok} - {matkul_bentrok}"
        )
        tampilkan_konflik(
            "Konflik Dosen", konflik_dosen,
            "Dosen: {dosen} | Hari: {hari} | Waktu: {waktu}\n    ↪ {kelas} - {matkul} ❌ bentrok dengan {kelas_bentrok} - {matkul_bentrok}"
        )
        tampilkan_konflik(
            "Konflik Kelas", konflik_kelas,
            "Kelas: {kelas} | Hari: {hari} | Waktu: {waktu}\n    ↪ {matkul} ❌ bentrok dengan {matkul_bentrok}"
        )

    def jumlah_jadwal_gagal(self, solution):
        return sum(1 for _, _, _, waktu in solution if waktu == (-1, -1))



sa = PenjadwalanSA()
sa.tampilkan_slot_waktu()
best_solution, best_score = sa.anneal()
print("Skor akhir:", best_score)
print("Jumlah jadwal gagal:", sa.jumlah_jadwal_gagal(best_solution), "dari total", len(sa.daftar_matkul))

df_jadwal = sa.df_jadwaloptimasi(best_solution) #pastikan df_jadwal memiliki id_perkuliahan

print(df_jadwal)
sa.cek_bentrok(best_solution)
# sa.simpan_optimasi(df_jadwal)
    

Daftar Slot Waktu:
1. 07:00 - 07:50
2. 07:50 - 08:40
3. 08:40 - 09:30
4. 09:45 - 10:35
5. 10:35 - 11:25
6. 11:25 - 12:15
7. 12:45 - 13:35
8. 13:35 - 14:25
9. 14:25 - 15:15
10. 15:30 - 16:20
11. 16:20 - 17:10
12. 17:10 - 18:00
13. 18:15 - 19:05
Iterasi 0 | Skor saat ini: 11010 | Skor terbaik: 11010 | Temperatur: 99999.0000
Iterasi 10000 | Skor saat ini: 235 | Skor terbaik: 130 | Temperatur: 90482.7917
Iterasi 20000 | Skor saat ini: 215 | Skor terbaik: 130 | Temperatur: 81872.1747
Iterasi 30000 | Skor saat ini: 200 | Skor terbaik: 130 | Temperatur: 74080.9701
Iterasi 40000 | Skor saat ini: 230 | Skor terbaik: 130 | Temperatur: 67031.2002
Iterasi 50000 | Skor saat ini: 205 | Skor terbaik: 130 | Temperatur: 60652.3078
Iterasi 60000 | Skor saat ini: 205 | Skor terbaik: 130 | Temperatur: 54880.4502
Iterasi 70000 | Skor saat ini: 245 | Skor terbaik: 130 | Temperatur: 49657.8600
Iterasi 80000 | Skor saat ini: 240 | Skor terbaik: 130 | Temperatur: 44932.2674
Iterasi 90000 | Skor saat ini: 180 |